In [3]:
import random
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import math

#可直接在K and N_F新增起火點和消防員的index
#設定Q b H為隨機產生或是給定
mode = "random"
filename1="fighter_route.xlsx"
filename2="fire_route.xlsx"
N_number=30
N = set([i for i in range(1,N_number+1)])  #所有點的集合
# T_number =200 
T_number = 200
T = list([i for i in range(T_number+1)])    #時間集合
K = set([1])                        #K=消防車集合
N_D = set([18])                      #Depot
N_F = set([5])                      #起火點
H={}
Q={}
b={}
#rand_fightertraveltime=[]
#rand_firetraveltime=[]
# H={1:4,2:3,3:6,4:1,5:2,6:2,8:1,9:6,10:5}
# Q={1:3,2:4,3:5,4:3,5:3,6:4,8:1,9:7,10:10}
# b={1:1,2:2,3:3,4:4,5:5,6:6,8:8,9:9,10:10}
P={1:3,2:4}                                  #各個消防車單位時間處理的燃料量

if mode == "random":
    for i in range(1,N_number+1):
        if i not in N_D:
            #Q[i] = random.randint(5,15)
            Q[i]=5
            #b[i] = random.randint(50,100)
            b[i]=30
            #H[i] = random.randint(5,15)
            H[i] = 5
else:
    H={1:1,2:2,3:1,4:2,5:4,6:3,7:3,8:3,9:2,10:2,11:2,12:4,13:2,15:3,16:6,17:5,18:2,19:1,20:4}                  #q=單位時間火在i點燒的燃料量
    Q={1:2,2:3,3:4,4:4,5:3,6:2,7:6,8:5,9:4,10:6,11:9,12:8,13:3,15:5,16:3,17:3,18:4,19:6,20:7}                  #h=火燒i點需要的時間
    b={1:40,2:40,3:30,4:50,5:70,6:30,7:70,8:40,9:40,10:50,11:60,12:50,13:40,15:60,16:20,17:60,18:20,19:40,20:80}#b=節點i的價值
       

M=10
A_p=gp.tuplelist()
A_f=gp.tuplelist()
tau=gp.tupledict()
lamb=gp.tupledict()
process={}
for k in K:
    process[k]={}
    for i in Q:
        process[k][i] = math.ceil(Q[i]*H[i]/P[k])


print(process)       
#print("fighter:",rand_fightertraveltime)
#print("fire:",rand_firetraveltime)
print("Q:",Q)
print("H:",H)
print("b:",b)



{1: {1: 9, 2: 9, 3: 9, 4: 9, 5: 9, 6: 9, 7: 9, 8: 9, 9: 9, 10: 9, 11: 9, 12: 9, 13: 9, 14: 9, 15: 9, 16: 9, 17: 9, 19: 9, 20: 9, 21: 9, 22: 9, 23: 9, 24: 9, 25: 9, 26: 9, 27: 9, 28: 9, 29: 9, 30: 9}}
Q: {1: 5, 2: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 19: 5, 20: 5, 21: 5, 22: 5, 23: 5, 24: 5, 25: 5, 26: 5, 27: 5, 28: 5, 29: 5, 30: 5}
H: {1: 5, 2: 5, 3: 5, 4: 5, 5: 5, 6: 5, 7: 5, 8: 5, 9: 5, 10: 5, 11: 5, 12: 5, 13: 5, 14: 5, 15: 5, 16: 5, 17: 5, 19: 5, 20: 5, 21: 5, 22: 5, 23: 5, 24: 5, 25: 5, 26: 5, 27: 5, 28: 5, 29: 5, 30: 5}
b: {1: 30, 2: 30, 3: 30, 4: 30, 5: 30, 6: 30, 7: 30, 8: 30, 9: 30, 10: 30, 11: 30, 12: 30, 13: 30, 14: 30, 15: 30, 16: 30, 17: 30, 19: 30, 20: 30, 21: 30, 22: 30, 23: 30, 24: 30, 25: 30, 26: 30, 27: 30, 28: 30, 29: 30, 30: 30}


In [4]:
df = pd.read_excel(filename1)
df_num = len(df.index)
for i in N:
    A_p.append((i,i))
for i in range(df_num):
    u = int(df.iloc[i]['i'])
    v = int(df.iloc[i]['j'])
    fighterIndex = df.iloc[i]['k']
    time = df.iloc[i]['travel time']
    tau[u,v,fighterIndex] = time
    #避免在多消防員時重複紀錄arc set
    if (u,v) not in A_p:
        A_p.append((u,v))
       
       
    
df = pd.read_excel(filename2)
df_num = len(df.index)
for i in range(df_num):
    u = int(df.iloc[i]['i'])
    v = int(df.iloc[i]['j'])
    time = df.iloc[i]['travel time']
    lamb[u,v] = time
    A_f.append((u,v))
#print(A_p)
#print(A_f)
# print(tau)
# print(lamb)

A_f_NEIGHBOR={}                      #A_f_NEIGHBOR=與點i相鄰的點
    
for l in N-N_D:                          #定義A_f_NEIGHBOR
    connect = A_f.select('*',l)
    A_f_NEIGHBOR[l]=[]
    for temp in connect:
        A_f_NEIGHBOR[l].append(temp[0])
                      
A_f_NEIGHBOR_T={}                    #A_f_NEIGHBOR_T=紀錄 t-hi-Lambda(i,j)>=0 且 與j點相鄰的i點

for j in N-N_D-N_F:                         #定義A_f_NEIGHBOR_T
    for t in T:
        A_f_NEIGHBOR_T[j,t]=[]
        for i in A_f_NEIGHBOR[j]:
            if t-H[i]-lamb[i,j]>=0:
                A_f_NEIGHBOR_T[j,t].append(i)
model=gp.Model("FIREFIGHTER")


x={}
w={}
u={}
u_bar={}
v={}
v_bar={}
epsilon=1e-3

#定義x[i,j,k,t]
for k in K:
    for t in T:
        for i in range(len(A_p)):
                x[A_p[i][0],A_p[i][1],k,t]=model.addVar(vtype='B',name="x[%d,%d,%d,%d]" %(A_p[i][0],A_p[i][1],k,t))

#定義w[i,k,t]
for k in K:
    for t in T:
        for i in N:
            w[i,k,t]=model.addVar(vtype='B',name="w[%d,%d,%d]" %(i,k,t))

#定義u[i,t]
u = model.addVars(N, T, vtype="B",name="u")

#定義u_bar[i,k,t]
u_bar = model.addVars(N, K, T, vtype="B", name="u_bar")

#定義v[i,t]
v = model.addVars(N, T, vtype="B", name="v")
        
#定義v_bar[i,t]
v_bar = model.addVars(N, T, vtype="B", name="v_bar")        
        
model.update()

for k in K:                        #原點flow blance
    model.addConstr(gp.quicksum(x[i,j,k,0] for i,j in A_p) <= 1)
    
#depart from depot
for O in N_D:
    connect = A_p.select(O,'*')
    for k in K:
        model.addConstr(gp.quicksum(x[i,j,k,0] for i, j in connect) == 1)
#depot can't be processed
model.addConstrs((u_bar[i,k,t]==0 for i in N_D for k in K for t in range(T_number+1)))

#flow balance
for k in K:
    for t in range(1,T_number):
        for j in N: 
            in_connect = A_p.select('*',j)
            out_connect = A_p.select(j,'*')
            temp=0 #in-degree
            temp += w[j,k,t-1] # t-1在j idle
            if j in N_D: #j in depot set會有u_bar，只是都為0
                temp+=u_bar[j,k,t]
            else: #j not in depot set若現在的t>process time，則會有u_bar
                if process[k][j] <= t:
                    temp += u_bar[j,k,t-process[k][j]]
            for m,n in in_connect:
                if m != n and tau[m,n,k] <= t: #若現在的t>travel time，則會有x
                    temp += x[m,n,k,t-tau[m,n,k]]
            model.addConstr(temp == gp.quicksum(x[n,w,k,t] for n,w in out_connect),name="flow") #in-degree = out-degree
model.addConstrs((u_bar[i,k,t] + w[i,k,t] == x[i,i,k,t] for i in N-N_D for k in K for t in range(T_number)))
model.addConstrs((w[s,k,t] == x[s,s,k,t]) for s in N_D for k in K for t in range(T_number))

#######################################################################################################################
model.addConstrs(u[i, t]+u_bar.sum(i, '*', t) <= 1 for i in N for t in T) #constrain 6
model.addConstrs(v[i, t] + u[i, t] == v[i, t+1] for i in N for t in T[0:-1]) #constrain 7
model.addConstrs(v_bar[i, t] + u_bar.sum(i, '*', t) == v_bar[i, t+1] for i in N for t in T[0:-1]) #constrain 9
model.addConstrs(u_bar.sum(s, '*', '*') == 0 for s in N_D) #constrain 11
model.addConstrs(u.sum(s, '*') == 0 for s in N_D) #constrain 12
        
#######################################################################################################################
    
for j in N-N_D-N_F:                            #火焰的延燒
    for t in range(T_number):
        if len(A_f_NEIGHBOR_T[j,t])==0:
            model.addConstr(u[j,t] == 0,name='test')
        else:
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) / M <= u[j,t] + v[j,t] + v_bar[j,t+1])
            model.addConstr(gp.quicksum(u[i,t-H[i]-lamb[i,j]] for i in A_f_NEIGHBOR_T[j,t]) >= u[j,t])
            
for i in N_F:                           #起火點起火
    model.addConstr(u[i,0]==1)
    
for i in N:                            #所有節點不會在t-1~t已經被燒
    model.addConstr(v[i,0]==0)
    
for i in N:                            #所有節點不會在t-1~t已經被澆水
    model.addConstr(v_bar[i,0]==0)
            
for k in K:                             #消防員不能去已經被燃燒的節點
    for t in T:
        for l in range(len(A_p)):
            if A_p[l][1] not in N_D:
                if A_p[l][0] ==  A_p[l][1]:
                    if t+2 <= T_number:
                        model.addConstr(M*(1-v[A_p[l][1],t+1]) >= x[A_p[l][0], A_p[l][1],k,t])
                    else:
                        model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0], A_p[l][1],k,t])
                elif t + tau[A_p[l][0],A_p[l][1],k] + 1 <= T_number:
                    model.addConstr(M*(1-v[A_p[l][1],t + tau[A_p[l][0],A_p[l][1],k]]) >= x[A_p[l][0],A_p[l][1],k,t])
                else:
                    model.addConstr(M*(1-v[A_p[l][1],T_number]) >= x[A_p[l][0],A_p[l][1],k,t])
                    
                    
# model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
#                    gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j),GRB.MINIMIZE)
model.setObjective(gp.quicksum(gp.quicksum(u[i,t] for t in T) * b[i] for i in N-N_D) + 
                   gp.quicksum(epsilon*x[i,j,k,t] for (i,j,k,t) in x if i!=j)+
                   gp.quicksum(epsilon*t*u_bar[i,k,t] for i in N-N_D for k in K for t in T),GRB.MINIMIZE)

model.optimize()
model.write('test.lp')
print("optimal value : ",model.ObjVal)
                    

Set parameter Username
Academic license - for non-commercial use only - expires 2024-07-23
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: 13th Gen Intel(R) Core(TM) i5-13500, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 73945 rows, 64320 columns and 258763 nonzeros
Model fingerprint: 0xca8ce198
Variable types: 0 continuous, 64320 integer (64320 binary)
Coefficient statistics:
  Matrix range     [1e-01, 1e+01]
  Objective range  [1e-03, 3e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 48359 rows and 33610 columns
Presolve time: 1.13s
Presolved: 25586 rows, 30710 columns, 104392 nonzeros
Variable types: 0 continuous, 30710 integer (30710 binary)
Found heuristic solution: objective 840.0260000
Deterministic concurrent LP optimizer: primal and dual simplex
Showing first log only...

Concurrent spin time: 0.00s

Solved with dual simplex

R

In [5]:
EPS=1.e-2
print("x:")
step=1
for k in K:
    print()
    print("消防員%d的路徑" % k)
    temp = [elem for elem in x if elem[2]==k]
    for (i,j,k,t) in temp:
        if x[i,j,k,t].X >EPS:
            #print("x[%d,%d,%d,%d]" %(i,j,k,t) , x[i,j,k,t].X)
            if i != j:
                print("在時刻 %d 從node%d 移動到 node%d" %(t,i,j)," ,travel time:",tau[i,j,k])
            else:            
                if u_bar[i,k,t].X == 1:
                    print("在時刻 %d 對node%d進行保護"%(t,i)," ,processing time:",process[k][i])
                else:
                    print("在時刻 %d 在node%d idle" %(t,i))

print("w:")
for (i,k,t) in w:
    if w[i,k,t].X>EPS:
        print("w[%d,%d,%d]" %(i,k,t) , w[i,k,t].X)

print("u:")
for (i,t) in u:
    if u[i,t].X>EPS:
        print("u[%d,%d]" %(i,t) , u[i,t].X)

print("u_bar:")
for (i,k,t) in u_bar:
    if u_bar[i,k,t].X>EPS:
        print("u_bar[%d,%d,%d]" %(i,k,t) , u_bar[i,k,t].X)

print("v:")
for (i,t) in v:
    if v[i,t].X>EPS:
        print("v[%d,%d]" %(i,t) , v[i,t].X)

print("v_bar:")
for (i,t) in v_bar:
    if v_bar[i,t].X>EPS:
        print("v_bar[%d,%d]" %(i,t) , v_bar[i,t].X) 

x:

消防員1的路徑
在時刻 0 從node18 移動到 node24  ,travel time: 12.0
在時刻 12 從node24 移動到 node28  ,travel time: 17.0
在時刻 29 從node28 移動到 node22  ,travel time: 11.0
在時刻 40 對node22進行保護  ,processing time: 9
在時刻 49 從node22 移動到 node6  ,travel time: 14.0
在時刻 63 對node6進行保護  ,processing time: 9
在時刻 72 從node6 移動到 node22  ,travel time: 14.0
在時刻 86 從node22 移動到 node28  ,travel time: 11.0
在時刻 97 對node28進行保護  ,processing time: 9
在時刻 106 從node28 移動到 node15  ,travel time: 19.0
在時刻 125 對node15進行保護  ,processing time: 9
在時刻 134 在node15 idle
在時刻 135 在node15 idle
在時刻 136 在node15 idle
在時刻 137 在node15 idle
在時刻 138 在node15 idle
在時刻 139 在node15 idle
在時刻 140 在node15 idle
在時刻 141 在node15 idle
在時刻 142 在node15 idle
在時刻 143 在node15 idle
在時刻 144 在node15 idle
在時刻 145 在node15 idle
在時刻 146 在node15 idle
在時刻 147 在node15 idle
在時刻 148 在node15 idle
在時刻 149 在node15 idle
在時刻 150 在node15 idle
在時刻 151 在node15 idle
在時刻 152 在node15 idle
在時刻 153 在node15 idle
在時刻 154 在node15 idle
在時刻 155 在node15 idle
在時刻 156 在node15 idle
在時刻 157 在node15 idle
在時刻 

In [6]:
df = pd.read_excel("G_30_seed120.xlsx")
NODE_POS = {}
x_list = []
y_list = []
for i in range(len(df)):
    NODE_POS[i+1] = (int(df.iloc[i][0]*2), int(df.iloc[i][1])*2)
    x_list.append(int(df.iloc[i][0]))
    y_list.append(int(df.iloc[i][1]))
print(NODE_POS)
print(type(30))
print(type(x_list[0]))

{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}
<class 'int'>
<class 'int'>


In [12]:
import json
data={}

print(NODE_POS)
# NODE_POS = {1:(30, 300), 2:(100, 100), 3:(500, 250), 4:(450, 600), 5:(1000, 150)}
print(NODE_POS)
data['NODE_POS'] = NODE_POS
data['N'] = list(N)
data['N_D'] = list(N_D)
data['N_F'] = list(N_F)
data['K'] = list(K)
data['A_p'] = list([str(i) for i in A_p])
data['A_f'] = list([str(i) for i in A_f])
data['tau'] = dict((str(i), tau[i]) for i in tau)
data['lamb'] = dict((str(i), lamb[i]) for i in lamb)
data['T'] = T
data['q'] = Q
data['b'] = b
data['p'] = P
data['h'] = H
# Q={1:6,2:2,3:4,5:6}                  #q=單位時間火在i點燒的燃料量
# b={1:18,2:8,3:12,5:18}               #b=節點i的價值
# P=3                                  #1號消防車單位時間處理的燃料量
# H={1:2,2:3,3:3,5:2}                  #h=火燒i點需要的時間

temp = {}
for (i, j, k, t) in x:
    temp[str((i, j, k, t))] = x[i, j, k, t].X
data['x'] = temp

temp = {}
for (k, t, i) in w:
    temp[str((k, t, i))] = w[k, t, i].X
data['w'] = temp

temp = {}
for (t, i) in u:
    temp[str((t, i))] = u[t, i].X
data['u'] = temp

temp = {}
for (t, i, k) in u_bar:
    temp[str((t, i, k))] = u_bar[t, i, k].X
data['u_bar'] = temp

temp = {}
for (t, i) in v:
    temp[str((t, i))] = v[t, i].X
data['v']  = temp

temp = {}
for (t, i) in v_bar:
    temp[str((t, i))] = v_bar[t, i].X
data['v_bar'] = temp
json_data = json.dumps(data)
with open("data.json", "w") as file:
    file.write(json.dumps(data))

{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}
{1: (514, 188), 2: (952, 634), 3: (588, 940), 4: (90, 792), 5: (222, 250), 6: (320, 738), 7: (940, 706), 8: (316, 128), 9: (286, 216), 10: (502, 508), 11: (114, 208), 12: (878, 700), 13: (94, 866), 14: (448, 384), 15: (804, 688), 16: (436, 382), 17: (544, 834), 18: (708, 86), 19: (28, 356), 20: (288, 778), 21: (606, 138), 22: (420, 576), 23: (480, 258), 24: (688, 280), 25: (994, 378), 26: (502, 802), 27: (350, 900), 28: (552, 540), 29: (236, 330), 30: (526, 818)}


In [ ]:
print(data['A_p'])